In [9]:
import pandas as pd
import numpy as np
import collections
import operator #
import matplotlib.pyplot as plt
import itertools
%matplotlib inline

In [10]:
#Load the dataset
data = pd.read_csv("/Users/Rohit/Desktop/Data/test.txt", sep="\t")
mydict = {}
data.shape

(3614506, 5)

In [11]:
data.head() #Check

,AnonID,Query,QueryTime,ItemRank,ClickURL
0,479,family guy,2006-03-01 16:01:20,NaN,NaN
1,479,also sprach zarathustra,2006-03-02 14:48:55,NaN,NaN
2,479,family guy movie references,2006-03-03 22:37:46,1.0,http://www.familyguyfiles.com
3,479,top grossing movies of all time,2006-03-03 22:42:42,1.0,http://movieweb.com
4,479,top grossing movies of all time,2006-03-03 22:42:42,2.0,http://www.imdb.com


In [12]:
queries = data.Query.unique()
len(queries) #No of unique queries

1244496

In [13]:
total = 0
for i in data.Query:
    if i in mydict: 
        mydict[i] += 1
    else: 
        mydict[i] = 1
    total+=1

In [14]:
len(mydict)

1244496

In [15]:
#Verifying dictionary is correct
sum =0
users =""
for k,v in mydict.items():
    sum+=v   
print(sum) #Should be same as shape[0]

3614506


In [16]:
#Find the 100 most common queries in this list
commonQueries = sorted(mydict.items(), key=lambda x:-x[1])[:100]

In [17]:
#Given the top choices, we're going to try to look at some patterns in common between the users 
#that used the most frequented query
#Linear reg wouldn't work because its very sensitive to outliers, which is common in modern data
#Noticed most mispelled/unclear queries don't get a click.

In [18]:
from difflib import SequenceMatcher

def similar(a, b):
    return round(SequenceMatcher(None, a, b).ratio(),2)

similar(queries[5],queries[6])
print(queries[5])
print(queries[6])

similar(queries[5],queries[6])
#simCheck(s1,s2)

car window decals
car window sponsor decals


0.81

In [ ]:
scoresheet = {}
for q1 in queries:
    for q2 in queries:
        score = similar(q1,q2)
        if score in scoresheet:
            scoresheet[score] += 1
        else: 
            print(score)
            scoresheet[score] = 1

1.0
0.24
0.54
0.1
0.2
0.22
0.17
0.14
0.19
0.0
0.15
0.21
0.13
0.18
0.25
0.27
0.23
0.11
0.16
0.33
0.08
0.32
0.28
0.26
0.12
0.07
0.29
0.06
0.3
0.31
0.42
0.09
0.35
0.05
0.5
0.37
0.36
0.43
0.38
0.64
0.41
0.4
0.34
0.44
0.48
0.04
0.03
0.53
0.46
0.45
0.01
0.39
0.67
0.47
0.58
0.61
0.59
0.02
0.55
0.52
0.56
0.73
0.6
0.57
0.8
0.62
0.69
0.63
0.74
0.72
0.49
0.95
0.51
0.75
0.83
0.76
0.7
0.65
0.78
0.77
0.9
0.84
0.71
0.87
